# Transform trackID files to artistID files

In [1]:
# Paths to relevant files
albumFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/albumData1.txt'
trackFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/trackData1.txt'
artistFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/artistData1.txt'
genreFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/genreData1.txt'


##mode = 'r' #r for read, w for write only, a for append, r+ for r+w 
#f1 = open(trackFile,mode) 
#f2 = open(artistFile,mode) 

# Files to use:
trainFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/trainIdx1.txt'
validationFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/validationIdx1.txt'
testFile = '../Webscope_C15/ydata-ymusic-kddcup-2011-track1/testIdx1.txt'

#dummyFile = 'dummy-2-users.txt'


#mode = 'r' #r for read, w for write only, a for append, r+ for r+w 
#trainF = open(trainFile,mode) 

# Open WRITE file 
#outputTrain = 'artistTrainIdx1.txt'
#modeOut = 'a' #r for read, w for write only, a for append, r+ for r+w 
#os.remove(outputFile) #TODO: Is this really ok structure to have? 
#trainOut = open(outputTrain,modeOut) 


In [2]:
# Functions borrowed from other files
# Checks if a file line is a user line
def isUserLine(line):
    lineSplit = line.split('|')
    tabSplit = line.split('\t')
    if len(lineSplit) == 2 and len(tabSplit) == 1:
        #Got a user line pattern
        return True 

#Counts the number of lines in a file
def fileLength(filePath):
    count = 0
    file = open(filePath,'r') 
    for line in file:
        count = count + 1        
    return count

#print('File length of trainFile test: %s' % (fileLength(trainFile) == 253801265))
print('File length of trackFile test: %s' % (fileLength(trackFile) == 507172))
print('File length of albumFile test: %s' % (fileLength(albumFile) == 88909))
print('File length of artistFile test: %s' % (fileLength(artistFile) == 27888))
print('File length of genreFile test: %s' % (fileLength(genreFile) == 992))



def createTypeObject(filePath,nrDataTracked):
    nrLines = fileLength(filePath)

    dataObject = [[0 for x in range(0,nrLines)] for y in range(0,nrDataTracked)]
    # Called: userData[3][39] 3: item index, 39: is raingIndex
    # Uses zero-indexing

    file = open(filePath,'r')
    lineNr = 0
    for line in file:
        lineSplit = line.split('|')
        if len(lineSplit) < nrDataTracked:
            print('ERROR: too few datapoints in original file')
            return None
        for i in range(0,nrDataTracked):
            dataObject[i][lineNr] = lineSplit[i].rstrip()
        
        lineNr = lineNr + 1
    
    file.close()
    return dataObject
#abum
#artist
#genre

typeObject1 = createTypeObject(trackFile,3)
print('Test: createTypeObject 1: %s' % (typeObject1[2][0] == '587636'))
print('Test: createTypeObject 2: %s' % (typeObject1[0][507172-1] == '624960'))

File length of trackFile test: True
File length of albumFile test: True
File length of artistFile test: True
File length of genreFile test: True
Test: createTypeObject 1: True
Test: createTypeObject 2: True


In [3]:
# Transform a trackID to artistID
# PRE: itemID is confirmed to be of type in targetFile
# If no match found, returns None
def trackToArtist(itemID,trackF):
    trackID = int(itemID)
    file = open(trackF,'r')
    
    for line in file:
        splitLine = line.split('|')
        if len(splitLine) < 3: 
            print('Error: Too little info about track')
            break
        lineID = int(splitLine[0].rstrip())
        artistID = splitLine[2].rstrip()
        
        if trackID == lineID:
            #Have found a match
            #Check if artist data exists
            if artistID == 'None':
                return None
            else:
                return int(artistID)
        elif trackID < lineID:
            #Because the files are sorted we can terminate when we pass the sought for value
            #print('test')
            #We should NOT end up here
            #print('Error, item is not a track')    
            return None 
    # Should never reach these lines
    print('Error: Nothing descisive found')
    return None
        

print('Test1: %s' % (trackToArtist('0',trackFile) == 587636))
print('Test2: %s' % (trackToArtist(7,trackFile)  == None))
print('Test3: %s' % (trackToArtist(80,trackFile)  == None))
print('Test4: %s' % (trackToArtist('624960',trackFile) == 380811))

# Check if item itemID is of the type specified by typeFile
# Assumption: Because the files are sorted we can terminate when we pass the sought for value

# idPosition is zero-indexed, 0 means same id is returned 
def typeCheckAndNewID(inputID,typeObject,idPosition):
    
    itemID = str(inputID)
    #print(itemID)
    #file = open(fileName,'r')
    
    if itemID in typeObject[0]:
        #Match found
        i = typeObject[0].index(itemID)
        newID = typeObject[idPosition][i]
        return (True,newID)
    else:
        return (False,None)
 

    
trackObject = createTypeObject(trackFile,3)
albumObject = createTypeObject(albumFile,1)
artistObject = createTypeObject(artistFile,1)
genreObject = createTypeObject(genreFile,1)

print('Test5: %s' % (typeCheckAndNewID('5',trackObject,2) == (True,'196558')))
print('Test6: %s' % (typeCheckAndNewID('10',trackObject,0) == (False,None)))
print('Test7: %s' % (typeCheckAndNewID('10',artistObject,0) == (True,'10')))
print('Test8: %s' % (typeCheckAndNewID('200',artistObject,0) == (False,None)))

Test1: True
Test2: True
Test3: True
Test4: True
Test5: True
Test6: True
Test7: True
Test8: True


In [8]:
import os
import time

def createArtistVersion(inputFile,trackFile,albumFile,artistFile,genreFile,outFile,maxNrUsers):
    
    trackObject = createTypeObject(trackFile,3)
    albumObject = createTypeObject(albumFile,2)
    artistObject = createTypeObject(artistFile,1)
    genreObject = createTypeObject(genreFile,1)
     
    nrUsers = 0
    file = open(inputFile,'r')
    
    if os.path.isfile(outFile):
        os.remove(outFile)    
    out = open(outFile,'a')   
    
    for line in file:
        if isUserLine(line):
            if nrUsers % 100 == 0:
                print('User nr: %s processed' % (nrUsers))
            if nrUsers >= maxNrUsers:
                break
            #Have a user meta line, no change needed
            #print('User line')
            out.write(line)
            nrUsers = nrUsers + 1
            
        elif line == '':
            #Endof file reached reached, should never get here
            print('Error: Empty line')
            return 0
        else:
            #NOT user, has to be a rating, itemType is unknown
            tabSplit = line.split('\t')
            itemID = tabSplit[0]
            
            
            (isGenre,gIndex) = typeCheckAndNewID(itemID,genreObject,0)
            if isGenre:
                # Genre found
                #print('Genre found')
                if gIndex != 'None': 
                    out.write(line)
                    #print(line)
            else: 
                (isArtist,newIndex) = typeCheckAndNewID(itemID,artistObject,0)
                
                if isArtist:
                    #item is an Artist no cange needed to the line
                    #print('Artist found')
                    if newIndex != 'None': 
                        out.write(line)
                        #print(line)
                else:
                    (isAlbum,artistID) = typeCheckAndNewID(itemID,albumObject,1)
                    #print('test')
                    if isAlbum:
                        # Album found
                        #print('Album found')
                        
                        #If NONE, do nothing, line is discarded
                        if artistID != 'None': 
                            tabSplit[0] = str(artistID)
                            artistLine = '\t'.join(tabSplit)
                            out.write(artistLine)   
                            #out.write(line)
                            #print(artistLine)
                        else:
                            # If artist is unknown, keep original data
                            out.write(line)
                    else:
                        (isTrack,artistID) = typeCheckAndNewID(itemID,trackObject,2)
                        if isTrack:
                            #We have a track to convert to artist
                            #print('Track found')
                            #artistID = trackToArtist(itemID,trackFile)
                            #artistID = 0
                            
                            #If NONE, do nothing, line is discarded
                            if artistID != 'None': 
                                tabSplit[0] = str(artistID)
                                artistLine = '\t'.join(tabSplit)
                                out.write(artistLine)
                            else:
                                out.write(line)
                                
                        else:
                            print('ERROR: Unknown item-type')

    
# To reverse the split: 
#'\t'.join(tSplit)

start_time = time.time()
#dummyFile


#trainFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/trainIdx1.txt'
#validationFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/validationIdx1.txt'
#testFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/testIdx1.txt'

inputFile = trainFile

nrUsers = 200000
createArtistVersion(inputFile,trackFile,albumFile,artistFile,genreFile,'../data/temp/artist_version/train_%s_users.txt' % (nrUsers),nrUsers)
print('Done')
end_time=time.time()-start_time
print(end_time)

# Times to run: 100 users =>  176 sec =  2.9 min
#              1000 users => 2238 sec = 37 min

User nr: 0 processed
User nr: 100 processed
User nr: 200 processed
User nr: 300 processed
User nr: 400 processed
User nr: 500 processed
User nr: 600 processed
User nr: 700 processed
User nr: 800 processed
User nr: 900 processed
User nr: 1000 processed
Done
2367.506469964981


In [9]:
list1 = []
list1.append([3])

In [10]:
list1

[[3]]